In [ ]:
from __future__ import absolute_import, division, print_function
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd ./drive/'My Drive'/1D_EMI_Datasets_Codes/Section_A_ML_vs_DeepLearning/training_datasets_notenooks/n18

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
raw_dataset = pd.read_csv('TrainStructuredData_n18_csv.csv', header=None)
dataset = raw_dataset.copy()
dataset.head()

In [ ]:
trainx = dataset.iloc[:,0:200]
trainy = dataset.iloc[:,200:204]
print(trainx.shape,trainy.shape)

In [ ]:
raw_testset = pd.read_csv('../test_datasets/TestData1kcsv.csv', header=None)
testset = raw_testset.copy()
testx = testset.iloc[:,0:200]
testy = testset.iloc[:,200:204]
print(testx.shape,testy.shape)

In [ ]:
raw_testset2 = pd.read_csv('../test_datasets/TestData1kcsvS2.csv', header=None)
val_testset = raw_testset2.copy()
val_testx = val_testset.iloc[:,0:200]
val_testy = val_testset.iloc[:,200:204]

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

In [ ]:
train_stats = trainx.describe()

In [ ]:
train_stats = train_stats.transpose()
train_stats

In [ ]:
normed_train_data = norm(trainx)
normed_test_data = norm(testx)
normed_val_data = norm(val_testx)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64*16, activation='relu', input_shape=[len(trainx.keys())]),    
    layers.Dense(64, activation='relu'),    
    layers.Dense(4)
  ])

  optimizer = tf.keras.optimizers.Adam(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
EPOCHS = 1000
history = model.fit(
  normed_train_data, trainy,
  epochs=EPOCHS, validation_data = (normed_val_data,val_testy), verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, testy, verbose=2)

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()
cc = test_predictions.reshape(1000,4)
np.savetxt("predictions_nn_v3.csv", cc, delimiter=",")
#
fig, axs = plt.subplots(2, 2,figsize=(10,10))
axs[0, 0].scatter(testy.iloc[:,3].values, cc[:,3])
axs[0, 0].set_title('$\epsilon_1$')
axs[0, 1].scatter(testy.iloc[:,2].values, cc[:,2])
axs[0, 1].set_title('$\epsilon_2$')
axs[1, 0].scatter(testy.iloc[:,1].values, cc[:,1])
axs[1, 0].set_title('$\epsilon_3$')
axs[1, 1].scatter(testy.iloc[:,0].values, cc[:,0])
axs[1, 1].set_title('$\epsilon_4$')
for ax in axs.flat:
    ax.set(xlabel='True Values', ylabel='Predictions')
    ax.set(xlim=[1,10], ylim=[1,10])
plt.savefig('tahminlerr8.png')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
#plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#history.history['loss'].to_csv('training_loss.csv')
#history.history['val_loss'].to_csv('testing_loss.csv')
np.savetxt("loss8.csv", history.history['loss'], delimiter=",")
np.savetxt("val_loss8.csv", history.history['val_loss'], delimiter=",")

In [ ]:
relative_error = (testy-test_predictions.reshape(1000,4))/testy
np.power(np.sum(np.power(relative_error,2), axis=0)/1000,0.5)